In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
import pytz
 
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# connect to the trade account specifying a password and a server
authorized=mt5.login(server="ICMarketsSC-Demo",login=     ,password="  ")
if authorized:
    account_info=mt5.account_info()
    if account_info!=None:
        # display trading account data 'as is'
        print(account_info)
        # display trading account data in the form of a dictionary
        print("Show account_info()._asdict():")
        account_info_dict = mt5.account_info()._asdict()
        for prop in account_info_dict:
            print("  {}={}".format(prop, account_info_dict[prop]))
        print()
 
        # convert the dictionary into DataFrame and print
        df=pd.DataFrame(list(account_info_dict.items()),columns=['property','value'])
        print("account_info() as dataframe:")
        print(df)
else:
    print("failed to connect to trade account 25115284 with password=gqz0343lbdm, error code =",mt5.last_error())
# shut down connection to the MetaTrader 5 terminal
#mt5.shutdown()

In [ ]:

while True:
    import time
    from datetime import datetime
    
    account_info=mt5.account_info()
    account_info_dict = mt5.account_info()._asdict()


    accinfo=pd.DataFrame(list(account_info_dict.items()),columns=['property','value'])
    equity=accinfo.loc[13, ['value']]
    #equity2 = (equity/0.05) * 5/100
    equity2 = equity * (0.5/100)
    lot = (equity2*0.01)/0.7
    
    #ser = pd.Series(lot, copy=False)
    lot_compound =round(lot.iloc[0], 2)
    #print(lot_compound)
    
    now = datetime.now()   
    y = now.year +1
    m = now.month    
    d = now.day 
    now = datetime.now()
    
    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' object in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime(y,m, d, tzinfo=timezone)
    # get 10 EURUSD H4 bars starting from 01.10.2020 in UTC time zone
    rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_H1, utc_from, 1000)
   
    #rates = mt5.copy_rates_from("EURUSD", mt5.TIMEFRAME_H1, now, 10000)
    data = pd.DataFrame(rates)
    data['time']=pd.to_datetime(data['time'], unit='s')
    data['hour00'] = data['time'].dt.hour
    
    #Data variables
    data['direction_change'] = np.where(((data['close'].shift(2)-data['close'].shift(1))*(data['close'].shift(1)-data['close'])<0), 1, 0)
    data['direction_changegroup']=data['direction_change'].rolling(60).sum()
    data['direction_up'] = np.where(((data['close']-data['close'].shift(1))>0), 1, 0)
    data['direction_down'] = np.where(((data['close']-data['close'].shift(1))<0), 1, 0)
    data['direction_upgroup'] = data['direction_up'].rolling(60).sum()
    data['direction_downgroup'] = data['direction_down'].rolling(60).sum()

    data['direction_upr'] = np.where(((data['close']-data['close'].shift(1))>0), data['tick_volume'], 0)
    data['direction_downr'] = np.where(((data['close']-data['close'].shift(1))<0), data['tick_volume'], 0)
    data['upsum'] = data['direction_upr'].rolling(60).sum()
    data['downsum'] = data['direction_downr'].rolling(60).sum()
    
    
    data['difference'] = (data['direction_up']*data['upsum'])-(abs(data['direction_down'])*data['downsum'])
   
  
        

    data['profitpr']= data['high'].rolling(1).max()
    data['stoploss'] = data['low'].rolling(1).min()


    
    symbol = "EURUSD"
    symbol_info = mt5.symbol_info(symbol)
    lot = lot_compound
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask #ask
    price2 = mt5.symbol_info_tick(symbol).bid  #bid
    deviation = 40
    
    
    request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": data['stoploss'].iloc[-2],
    "tp": data['profitpr'].iloc[-2],
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}
    
    request2 = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,
    "price": price2,
    "sl": data['profitpr'].iloc[-2] ,
    "tp": data['stoploss'].iloc[-2] ,
    "deviation": deviation,
    "magic": 234001,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,
}  
    
    
    
    
    def close_position(position):

        tick = mt5.symbol_info_tick(position.symbol)

        request3 = {
            "action": mt5.TRADE_ACTION_DEAL,
            "position": position.ticket,
            "symbol": position.symbol,
            "volume": position.volume,
            "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
            "price": tick.ask if position.type == 1 else tick.bid,  
            "deviation": 20,
            "magic": 100,
            "comment": "python script close",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result3 = mt5.order_send(request3)
        return result3

    positions = mt5.positions_get()
    positions    
       
                
    if  (now.minute==0 and now.second ==0 and data['difference'].iloc[-2]>0 and (data['close'].iloc[-2] -data['close'].iloc[-3]>0))    : 
      #  for position in positions:
        #    close_position(position)
        result2 = mt5.order_send(request2)
        print(result2)
        time.sleep(3540)              
    elif (now.minute==0 and now.second ==0 and data['difference'].iloc[-2]<0 and (data['close'].iloc[-2] -data['close'].iloc[-3]<0))   :
        result = mt5.order_send(request)
        print(result) 
        time.sleep(3540)  
